In [2]:
import numpy as np
import math
import random
import matplotlib.pyplot as plt

In [3]:
m = 10 # количество частей файла (скачиваются по порядку. У лича >= 1 части, у сида все m частей)
n, l = m + 1, m + 2 # количество видов узлов и возможных взаимодействий между ними

lichers = np.array([m if i == 0 or i == m else i for i in range(m + 1)]) # количество личей с i частями файла
lichers_ = np.append(np.cumsum(lichers[::-1] 
            )[:-1:][::-1], 0) # количество подходящих для лича узлов для передачи файла (для новых клиентов - все личи + сиды)
alpha = np.append(lichers, lichers_)

lmb = 2
delta = sum(lichers_)
mu = 2
phi = ([lmb] +
       [lichers_[i] * lichers[i] / delta for i in range(m)] +
       [mu])

eps = np.array([[0 for i in range(m + 1)] + [0 for i in range(m + 1)]] +
               [[1 if i == j else 0 for i in range(m + 1)] + [1 if i == j else 0 for i in range(m + 1)] for j in range(m)] +
               [[1 if i == m else 0 for i in range(m + 1)] + [0 for i in range(m + 1)]])

p_gamma = np.array([[1 if i == 0 else 0 for i in range(m + 1)] + [0 for i in range(m + 1)]] +
                   [[1 if i == j + 1 else 0 for i in range(m + 1)] + [1 if i <= j else 0 for i in range(m + 1)] for j in range(m)] +
                   [[0 for i in range(m + 1)] + [0 for i in range(m + 1)]])

In [4]:
def inverse_exp(random_, coef):
    '''Возвращает значение случайной величины,
    распределённой по экспоненциальному закону,
    соответствующему простейшему Пуассоновскому потоку'''
    if coef == 0:
        return np.finfo(np.float32).max
    return -1/coef * math.log(1 - random_)

tau = np.array([0 for _ in range(l)], dtype=np.float32) # время следующего события i
for i in range(l):
    tau[i] = inverse_exp(random.random(), phi[i])

def update_tau(tau, ):
    ''' Возвращает матрицу с временами событий на следующем шаге и ближайшее событие
    :param tau: array of trigger times, generated on the previous step
    :return: times array on the next step, next event, time of next event'''
    min_ = np.finfo(np.float32).max
    idx_min = 0

    for k in range(len(tau)):
        if tau[k] == np.finfo(np.float32).max and phi[k] != 0:
            tau[k] = inverse_exp(random.random(), phi[k])
        if tau[k] < min_:
            min_ = tau[k]
            idx_min = k
    tau -= min_
    return tau, idx_min, min_

def update_lichers_(lichers):
    return np.append(np.cumsum(lichers[::-1] 
            )[:-1:][::-1], 0)

def update_phi(alpha):
    phi = np.array([lmb] +
                   [lichers_[i] * lichers[i] / delta for i in range(m)] +
                   [mu])
    phi[~np.all(eps <= alpha, axis=1)] = 0
    return phi

def taking_action(alpha, next_event):
    """Совершаем следующее предопределённое действие
    :param alpha: array of existing particles
    :param next_event: next event with prerequisites from eps and consequences determined from p_gamma
    :return: next state and new probabilities of events"""
    alpha += -eps[next_event] + p_gamma[next_event]
    phi = update_phi(alpha)
    return alpha, phi

In [5]:
print(tau)
print(eps)
print(p_gamma)
print(alpha)

[0.4137897  0.38238502 0.38321894 2.3231246  0.08887681 3.1620188
 0.09870549 0.32251227 2.7381177  4.1950936  0.249065   0.94329864]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]]
[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0